<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div>  
<br><br><br><br>
<br>

# Simulation of dose-response cruves of agonsits using data acquired with tauRAMD 


In this notebook we will a simulate mathematical model of a signaling pathway to obtain dose-response curves using kinetic values obtained with the <b>tauRAMD</b> tool.

<b>tauRAMD</b> is a tool developed by *Kokh et al.*:
<blockquote>

Estimation of Drug-Target Residence Times by τ-Random Acceleration Molecular Dynamics Simulations
Daria B. Kokh, Marta Amaral, Joerg Bomke, Ulrich Grädler, Djordje Musil, Hans-Peter Buchstaller, Matthias K. Dreyer, Matthias Frech, Maryse Lowinski, Francois Vallee, Marc Bianciotto, Alexey Rak, and Rebecca C. Wade
Journal of Chemical Theory and Computation 2018 14 (7), 3859-3869
DOI: 10.1021/acs.jctc.8b00230
 
<a href="https://doi.org/10.1021/acs.jctc.8b00230" target="_bank"> https://doi.org/10.1021/acs.jctc.8b002308</a>
</blockquote>

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/SSBtoolkit-Tutorial3B-tauRAMD.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>
</div>

## Installing the SSB toolkit library

In [ ]:
!pip install ssbtoolkit

### Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import ssbtoolkit as ssb

## Calculate residence times with tauRAMD

To calculate residence times with tauRAMD we have to have the RAMD simulations already performed.

In this tutorial we will use the pre-performed  RAMD simulations (4 replicas with 15 trajectories per replica) for 2 different agonists of the Adenosine A2A receptor:
- Adenosine
- Neca

The times.dat files can been found in `examples/tauRAMD/A2A/`.


In [ ]:
%%bash
if [ ! -d "SSBtoolkit/" ]; then
  git clone https://github.com/rribeiro-sci/SSBtoolkit.git --quiet
fi

In [ ]:
# we will start by creating a simulation instance for different ligands.

tauADN = ssb.Utils.tauRAMD()
tauNEC = ssb.Utils.tauRAMD()

In [ ]:
# we can run the tRAMD for each ligand
tauADN.Run(prefix='SSBtoolkit/example/tauRAMD/A2A/times_ADN')
tauNEC.Run(prefix='SSBtoolkit/example/tauRAMD/A2A/times_NEC')

The representation of the bootstrapping output and statistics are accessible through the functions:
- `Utils.tauRAMD.PlotRTStats()`
- `Utils.tauRAMD.PlotRTDistribuitons()`
    

In [ ]:
tauADN.PlotRTStats()

In [ ]:
tauADN.PlotRTDistribuitons()

The residence times from each replica can be accessed by the attibute `RTdataframe`:

In [ ]:
tauADN.RTdataframe

and the average residence time can be accessed by the attribute `RT`:

In [ ]:
tauADN.RT

The residence time can be converted in k<sub>off</sub> by:

$k_{off}=\frac{1}{RT}$

Let's create a  python dictonary with the calulated k<sub>off</sub> values fro each ligand:

In [ ]:
ligands = ['ADN', 'NEC']

kinetic_parameters=[{'RL_koff':1/tauADN.RT},{'RL_koff':1/tauNEC.RT}]

##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, individual simulations of the pathway according to a series of ligand concentrations must be performed (as it would be done in the wet-lab).  

To define an array of ligand concentrations we will use a geometric progression. The reason why we use a geometric progression is due to the impact of the dilution fraction on the accuracy of K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values experimentally estimated. This factor, that defines the spacing between the adjacent concentration values, has an impact on the concentration values that are on the linear portion of the curve. Therefore, using a geometric progression we can mimic the experimental conditions where each concentration equals to the power of 2 of the previous lowest concentration *([Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426))*

<span style="color:black"> ⚠️ WARNING: the SSB toolkit uses μM as default concentration units.</span>


In [ ]:
# setting the range of ligand concentration
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

# Setting receptor concentration
receptor_conc = 1E-3 #μM

# defining other simulation parameters:
time = 10000  # time of simulation in seconds
nsteps = 1000 # number of time steps


## Selection of the signaling pathway 

The core of the SSB framework is, naturally, the mathematical models of the GPCRs' signaling pathways. 

Since G-protein sub-families are classified by their $\alpha$ subunits, this classfication as been served to identify their signaling pathways:
* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 See [The SSB toolkit](https://ssbtoolkit.readthedocs.io) documentation for more details.

We can define manualy the G$\alpha$ pathway we want to work with, or simply query our internal database of human GPCR pathways using the UNIPROT id of our receptor using the SSBtoolkit built-in function `Utils.GetGProtein()`. The UNIPROT id for the human Adenosine A2 receptot is [P29274](https://www.uniprot.org/uniprot/P29274).

In [ ]:
# Selecting the signaling pathway 
uniprotID = 'P29274' # Adenosine A2 receptor Uniprot ID
gprotein = ssb.Utils.GetGProtein(uniprotID)
gprotein


## Integration of ODEs 

After having defined all the simulation parameters we are ready to proceed with the simulations. A simulation of a methamatical model of a signaling pathway consists of the integration of a set of ordinary differential equations (ODEs) as function of time. Since each ODE represents a molecular event of the signaling pathway, when integrated over time, such equations can describe how the concentration of species inside the model changes over time.

Because we are using kinetic values we have to set `kinetics=True` in the `Simulation.Activation.SetSimulationParameters()` instance. If we use option `kinetcs=True`, we also need to pass to the instance a dictionary of parameters, the one we had prepared before. 


In [ ]:
#create a simulation instance.
sim = ssb.Simulation.Activation()

In [ ]:
#Setting the simulation parameters
sim.SetSimulationParameters(ligands=ligands, 
                            pathway=gprotein, 
                            receptor_conc=receptor_conc, 
                            lig_conc_range=lig_conc_range, 
                            ttotal=time, 
                            nsteps=nsteps,  
                            binding_kinetics=True, 
                            binding_kinetic_parameters=kinetic_parameters)


In [ ]:
#Running the simulation
sim.Run()

In the end, the concentration values of the species of the signaling pathway over the simulation time will be saved inside the instance.

The response of a signaling pathway is, naturally, represented by the increase or decrease of one of the species described by the model. So, to predict the dose-response curve we need, firstly, to extract the maximum concentration value orbserved for one specie from each individual simulation (from the series of simulations for each ligand concentration). Then, such values will be fitted to a logistic regression. 

To achieve this, we will use the analysis attribute:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves we can interpolate the EC<sub>50</sub> values.

In [ ]:
sim.ShowPotency()

💡 The potency predicted values can be exported as a python dictionary using the function `sim.PotencyToDict()` or saved in a csv file: `sim.PotencyToCSV()`. 

## Congratulations! 

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with SSBtoolkit, we encourage you to finish the rest of the tutorials in this series. 

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={{SSB} toolkit: from molecular structure to subcellular signaling pathways.},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={bioRxiv},
    url={https://www.biorxiv.org/content/10.1101/2022.11.08.515595v1},
    doi={10.1101/2022.11.08.515595},
    year={2022}
}
```


## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  